<a href="https://colab.research.google.com/github/HimanshuKumar05/food-label-compliance-checker/blob/main/foodLabelComplianceChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re

# Sample dataset with food label information (replace this with a real dataset)
data = {
    'product_name': ['Chips', 'Cookies', 'Juice'],
    'ingredients': ['Potatoes, Salt, Oil', 'Wheat, Sugar, Butter', 'Water, Sugar, Mango Pulp'],
    'nutrition_facts': ['Calories: 500, Fat: 30g, Sugar: 5g, Salt: 100mg',
                        'Calories: 400, Fat: 20g, Sugar: 25g, Salt: 200mg',
                        'Calories: 150, Fat: 1g, Sugar: 35g, Salt: 50mg'],
    'health_claims': ['Low fat', 'Sugar-free', 'Rich in fiber']
}

df = pd.DataFrame(data)

# Define FSSAI compliance rules
fssai_rules = {
    "ingredients": ["potatoes", "wheat", "salt", "oil", "sugar", "mango pulp", "butter"],
    "nutrition_facts": {
        "calories_max": 500,
        "fats_max": 20,
        "sugar_max": 30,
        "salt_max": 150  # In mg
    },
    "allowed_health_claims": ["low fat", "rich in fiber", "no added sugar"]
}


In [2]:
# Function to clean and standardize text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Preprocess ingredients and nutrition facts
df['ingredients_clean'] = df['ingredients'].apply(lambda x: clean_text(x).split(', '))
df['nutrition_facts_clean'] = df['nutrition_facts'].apply(lambda x: clean_text(x))


In [3]:
# Function to check compliance based on FSSAI rules
def check_compliance(row):
    # Check ingredients
    non_compliant_ingredients = [ingredient for ingredient in row['ingredients_clean'] if ingredient not in fssai_rules['ingredients']]

    # Check nutrition facts
    nutrition_issues = []
    facts = row['nutrition_facts_clean']

    calories = int(re.search(r'calories (\d+)', facts).group(1))
    if calories > fssai_rules['nutrition_facts']['calories_max']:
        nutrition_issues.append(f"Calories exceed limit: {calories}")

    fat = int(re.search(r'fat (\d+)g', facts).group(1))
    if fat > fssai_rules['nutrition_facts']['fats_max']:
        nutrition_issues.append(f"Fats exceed limit: {fat}g")

    sugar = int(re.search(r'sugar (\d+)g', facts).group(1))
    if sugar > fssai_rules['nutrition_facts']['sugar_max']:
        nutrition_issues.append(f"Sugar exceeds limit: {sugar}g")

    salt = int(re.search(r'salt (\d+)mg', facts).group(1))
    if salt > fssai_rules['nutrition_facts']['salt_max']:
        nutrition_issues.append(f"Salt exceeds limit: {salt}mg")

    # Check health claims
    health_claim = clean_text(row['health_claims'])
    if health_claim not in fssai_rules['allowed_health_claims']:
        return "Non-compliant health claim: " + health_claim

    # Combine all issues
    issues = non_compliant_ingredients + nutrition_issues
    if len(issues) == 0:
        return "Compliant"
    else:
        return ", ".join(issues)

# Apply the compliance check on the dataset
df['compliance_status'] = df.apply(check_compliance, axis=1)
df[['product_name', 'compliance_status']]


,product_name,compliance_status
0,Chips,"potatoes salt oil, Fats exceed limit: 30g"
1,Cookies,Non-compliant health claim: sugarfree
2,Juice,"water sugar mango pulp, Sugar exceeds limit: 35g"


In [4]:
print(df[['product_name', 'compliance_status']])


  product_name                                 compliance_status
0        Chips         potatoes salt oil, Fats exceed limit: 30g
1      Cookies             Non-compliant health claim: sugarfree
2        Juice  water sugar mango pulp, Sugar exceeds limit: 35g
